In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
import pandas as pd
from collections import defaultdict, Counter
import re
from math import log
from math import exp
from math import log, exp
import csv
from collections import defaultdict
from math import log, exp
import re
import re
from collections import defaultdict
from math import log, exp
from sklearn.metrics import accuracy_score

## References

code for Dataset Reference for the Generated Text :
https://www.kaggle.com/code/ravindra1010/naive-bayes

Baisc Idealogy for the Naive Bayes using words and class :
https://github.com/parham1998/Naive-Bayes-Classifier/blob/main/naive_bayes.py 

Basic Reference for Naive Bayes classification and how to add Smoothing 
https://sebastianraschka.com/Articles/2014_naive_bayes_1.html

In [17]:
training_data  = pd.read_csv('C:/Users/Chinmay/Downloads/Generated_text/Generated_text.csv')
training_dataset  = pd.read_csv('C:/Users/Chinmay/Downloads/llm-detect-ai-generated-text/train_essays.csv').to_dict('records')
test_data  = pd.read_csv('C:/Users/Chinmay/Downloads/llm-detect-ai-generated-text/test_essays.csv')
train_prompts_data   = pd.read_csv('C:/Users/Chinmay/Downloads/llm-detect-ai-generated-text/train_prompts.csv')

In [18]:
train_ratio = 0.8
train_size = int(len(training_data) * train_ratio)

train_dataset = training_data[:train_size]
dev_dataset = training_data[train_size:]

In [19]:
text = train_dataset['text']
labels = train_dataset['label']

In [20]:
from collections import Counter

def build_vocabulary(data, min_occurrences=5):
    all_words = [word for essay in data['text'] for word in essay.split()]
    word_counts = Counter(all_words)
    
    rare_words = [word for word, count in word_counts.items() if count < min_occurrences]
    
    vocabulary = list(set(all_words) - set(rare_words))
    
    reverse_index = {word: index for index, word in enumerate(vocabulary)}
    
    return vocabulary, reverse_index

vocabulary, reverse_index = build_vocabulary(train_dataset)


# Reference:

Reference for word Count for building naive bayes classifier :
https://www.kaggle.com/code/antmarakis/word-count-and-naive-bayes

In [21]:
def train_naive_bayes_classifier(documents, labels, alpha):
    class_probs = defaultdict(float)
    word_probs = defaultdict(lambda: defaultdict(float))
    classes = set()

    class_counts = defaultdict(int)
    for label in labels:
        class_counts[label] += 1
        classes.add(label)

    total_documents = len(labels)

    for label, count in class_counts.items():
        class_probs[label] = count / total_documents

    word_counts = defaultdict(lambda: defaultdict(int))

    for doc, label in zip(documents, labels):
        words = re.findall(r'\b\w+\b', doc.lower())
        for word in words:
            word_counts[label][word] += 1

    for label in classes:
        total_words_in_class = sum(word_counts[label].values())
        total_unique_words = len(set(word_counts[label].keys()))

        for word, count in word_counts[label].items():
            word_probs[label][word] = (count + alpha) / (total_words_in_class + alpha * total_unique_words)

    return class_probs, word_probs, classes

def predict_naive_bayes_classifier(documents, class_probs, word_probs, classes):
    predictions = []

    for document in documents:
        words = re.findall(r'\b\w+\b', str(document).lower())
        scores = defaultdict(float)

        for label in classes:
            scores[label] = log(class_probs[label])

            for word in words:
                scores[label] += log(word_probs[label].get(word, 1e-10))

        exp_scores = {label: exp(score) for label, score in scores.items()}
        sum_exp_scores = sum(exp_scores.values())

        if sum_exp_scores == 0:
            probabilities = {label: 1 / len(classes) for label in classes}
        else:
            probabilities = {label: exp_score / sum_exp_scores for label, exp_score in exp_scores.items()}

        predictions.append(probabilities)

    return predictions



In [22]:
dev_data_text = dev_dataset['text']
dev_data_labels = dev_dataset['label']

In [23]:
alpha_values_to_try = [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
for alpha in alpha_values_to_try:
    class_probs, word_probs, classes = train_naive_bayes_classifier(text, labels, alpha)
    predictions = predict_naive_bayes_classifier(dev_data_text, class_probs, word_probs, classes)
    result=[]
    for i in range(len(predictions)):
        if(predictions[i][0] >= predictions[i][1]):
            result.append(0)
        else:
            result.append(1)
    count = 0
    for i in range(len(result)):
        if(result[i] == dev_data_labels.iloc[i]):
            count = count + 1

    accuracy = count / len(result)
    print("For alpha value:", alpha, "Accuracy:", accuracy)

For alpha value: 0.01 Accuracy: 0.15856919991085358
For alpha value: 0.1 Accuracy: 0.15856919991085358
For alpha value: 0.5 Accuracy: 0.15856919991085358
For alpha value: 1.0 Accuracy: 0.15856919991085358
For alpha value: 5.0 Accuracy: 0.15856919991085358
For alpha value: 10.0 Accuracy: 0.15856919991085358


In [24]:
from collections import Counter

def calculate_probabilities(data, vocabulary):
    total_documents = len(data)
    word_counts = Counter(' '.join([row['text'] for row in data]).split())
    occurrence_probability = {word: word_counts[word] / total_documents for word in vocabulary}

    llm_documents = [row['text'] for row in data if row['generated'] == 1]
    llm_word_counts = Counter(' '.join(llm_documents).split())
    conditional_probability_llm = {word: llm_word_counts[word] / len(llm_documents) for word in vocabulary}

    return occurrence_probability, conditional_probability_llm



In [25]:
occurrence_probability, conditional_probability_llm = calculate_probabilities(training_dataset, vocabulary)

In [26]:
top_10_llm_words = sorted(vocabulary, key=lambda word: conditional_probability_llm[word], reverse=True)[:10]
top_10_human_words = sorted(vocabulary, key=lambda word: occurrence_probability[word], reverse=True)[:10]

print("Top 10 words predicting LLM essays:", top_10_llm_words)
print("Top 10 words predicting human essays:", top_10_human_words)

Top 10 words predicting LLM essays: ['the', 'of', 'to', 'for', 'a', 'and', 'that', 'votes', 'in', 'is']
Top 10 words predicting human essays: ['the', 'to', 'of', 'a', 'and', 'in', 'is', 'that', 'for', 'are']


In [27]:
test_text = test_data['text']
print(test_text)
testing_data_probabilities = predict_naive_bayes_classifier(test_text, class_probs, word_probs, classes)
print(testing_data_probabilities)

0    Aaa bbb ccc.
1    Bbb ccc ddd.
2    CCC ddd eee.
Name: text, dtype: object
[{0: 0.6704230005462533, 1: 0.3295769994537467}, {0: 0.7242436061737354, 1: 0.2757563938262646}, {0: 0.7242436061737354, 1: 0.2757563938262646}]


In [28]:
final_testing_output = []
for i in testing_data_probabilities:
    final_testing_output.append(i[1])
predicted_output = np.array(final_testing_output)

In [29]:
submission= pd.DataFrame({'id':test_data["id"],'generated':predicted_output})
submission.to_csv('submission.csv', index=False)
print(submission)

         id  generated
0  0000aaaa   0.329577
1  1111bbbb   0.275756
2  2222cccc   0.275756


In [30]:
import nltk

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

# Download NLTK resources
import nltk
nltk.download('stopwords')
class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = defaultdict(float)
        self.word_probs = defaultdict(lambda: defaultdict(float))
        self.classes = set()

    def train(self, documents, labels, alpha):
        # Count occurrences of each class
        class_counts = defaultdict(int)
        for label in labels:
            class_counts[label] += 1
            self.classes.add(label)

        total_documents = len(labels)

        # Calculate class probabilities
        for label, count in class_counts.items():
            self.class_probs[label] = count / total_documents

        # Count word occurrences in each class with Lidstone smoothing
        for doc, label in zip(documents, labels):
            words = doc.lower().split()
            for word in words:
                self.word_probs[label][word] += 1

        # Calculate word probabilities for each class with Lidstone smoothing
        for label in self.classes:
            total_words_in_class = sum(self.word_probs[label].values())
            total_unique_words = len(self.word_probs[label])

            for word, count in self.word_probs[label].items():
                self.word_probs[label][word] = (count + alpha) / (total_words_in_class + alpha * total_unique_words)

    def predict(self, documents):
        # Assuming 'documents' is a list of documents
        probabilities_list = []

        for document in documents:
            words = document.lower().split()
            scores = defaultdict(float)

            for label in self.classes:
                scores[label] = log(self.class_probs[label])

                for word in words:
                    scores[label] += log(self.word_probs[label].get(word, 1e-10))

            exp_scores = {label: exp(score) for label, score in scores.items()}
            sum_exp_scores = sum(exp_scores.values())

            if sum_exp_scores == 0:
                probabilities = {label: 1 / len(self.classes) for label in self.classes}
            else:
                probabilities = {label: exp_score / sum_exp_scores for label, exp_score in exp_scores.items()}

            probabilities_list.append(probabilities)

        return probabilities_list

# Create a function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Remove stopwords and apply stemming
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    
    return ' '.join(words)

# Assuming 'train_data' is your training dataset
train_dataset['processed_text'] = train_dataset['text'].apply(preprocess_text)

# Split the data into train and validation sets

# Create and train the classifier with Lidstone smoothing
alpha = 0.1
classifier = NaiveBayesClassifier()
classifier.train(text, labels, alpha)

# Make predictions on the development set
dev_predictions = classifier.predict(dev_data_text)

# Assuming 'dev_labels' is your actual labels for the development set
dev_result = [max(prediction, key=prediction.get) for prediction in dev_predictions]
dev_accuracy = accuracy_score(dev_data_labels, dev_result)

print("Development Accuracy:", dev_accuracy)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chinmay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Chinmay\AppData\Local\Temp\ipykernel_18300\3461143589.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['processed_text'] = train_dataset['text'].apply(preprocess_text)


Development Accuracy: 0.15723200356585693


## Reference : 
https://www.analyticsvidhya.com/blog/2022/03/building-naive-bayes-classifier-from-scratch-to-perform-sentiment-analysis/

Code reference for TDF IDF Vecortizer :
https://www.analyticsvidhya.com/blog/2022/03/building-naive-bayes-classifier-from-scratch-to-perform-sentiment-analysis/


## Experimentation for Improvization using  TF-IDF
Instead of using a simple word count, using the TF-IDF (Term Frequency-Inverse Document Frequency) representation. TF-IDF takes into account the importance of words in a document relative to their frequency across all documents.

In [33]:
import re
from collections import defaultdict
from math import log, exp
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = defaultdict(float)
        self.word_probs = defaultdict(lambda: defaultdict(float))
        self.classes = set()

    def train(self, documents, labels, alpha, vectorizer):
        # Count occurrences of each class
        class_counts = defaultdict(int)
        for label in labels:
            class_counts[label] += 1
            self.classes.add(label)

        total_documents = len(labels)

        # Calculate class probabilities
        for label, count in class_counts.items():
            self.class_probs[label] = count / total_documents

        # Use TF-IDF representation
        tfidf_matrix = vectorizer.transform(documents)

        # Count word occurrences in each class with Laplace smoothing
        for i, label in enumerate(labels):
            for word, index in vectorizer.vocabulary_.items():
                self.word_probs[label][word] += tfidf_matrix[i, index]

        # Calculate word probabilities for each class with Laplace smoothing
        for label in self.classes:
            total_words_in_class = sum(self.word_probs[label].values())
            total_unique_words = len(self.word_probs[label])

            for word, count in self.word_probs[label].items():
                self.word_probs[label][word] = (count + alpha) / (total_words_in_class + alpha * total_unique_words)

    def predict(self, documents, vectorizer):
        # Assuming 'documents' is a list of documents
        probabilities_list = []

        # Use TF-IDF representation
        tfidf_matrix = vectorizer.transform(documents)

        for i in range(len(documents)):
            scores = defaultdict(float)

            for label in self.classes:
                scores[label] = log(self.class_probs[label])

                for word, index in vectorizer.vocabulary_.items():
                    scores[label] += log(self.word_probs[label].get(word, 1e-10)) * tfidf_matrix[i, index]

            exp_scores = {label: exp(score) for label, score in scores.items()}
            sum_exp_scores = sum(exp_scores.values())

            if sum_exp_scores == 0:
                probabilities = {label: 1 / len(self.classes) for label in self.classes}
            else:
                probabilities = {label: exp_score / sum_exp_scores for label, exp_score in exp_scores.items()}

            probabilities_list.append(probabilities)

        return probabilities_list


# Assuming 'train_data' is your training dataset
documents = [row['text'] for row in training_dataset]
labels = [row['generated'] for row in training_dataset]

# Fit the vectorizer on the training data
vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(documents)

# Create and train the classifier with TF-IDF
alpha = 0.1
classifier = NaiveBayesClassifier()
classifier.train(documents, labels, alpha, vectorizer)

# Assuming 'dev_dataset' is your development dataset and it's a Pandas DataFrame
dev_documents = dev_dataset['text'].tolist()

# Make predictions on the development set
predictions = classifier.predict(dev_documents, vectorizer)


In [36]:
print(predictions)

[{0: 0.9999970546521627, 1: 2.94534783729152e-06}, {0: 0.9999998155017402, 1: 1.8449825986201318e-07}, {0: 0.9998610395449512, 1: 0.00013896045504887882}, {0: 0.9999998014294812, 1: 1.9857051872327949e-07}, {0: 0.999999998029054, 1: 1.970945951626257e-09}, {0: 0.9999995620837392, 1: 4.379162607906973e-07}, {0: 0.9999896358046954, 1: 1.0364195304626339e-05}, {0: 0.9999932080982675, 1: 6.791901732532335e-06}, {0: 0.9999997345558496, 1: 2.654441504283394e-07}, {0: 0.999997066844928, 1: 2.9331550720004314e-06}, {0: 0.9999848661823593, 1: 1.5133817640756815e-05}, {0: 0.9999999970474771, 1: 2.9525228800596315e-09}, {0: 0.9999897713625728, 1: 1.0228637427182758e-05}, {0: 0.9999994964225678, 1: 5.035774322184661e-07}, {0: 0.9999942177026362, 1: 5.7822973638653555e-06}, {0: 0.9999997849893778, 1: 2.1501062221243147e-07}, {0: 0.9999975177495954, 1: 2.4822504044632686e-06}, {0: 0.9999999830638602, 1: 1.693613975941504e-08}, {0: 0.9999998498248066, 1: 1.5017519339614793e-07}, {0: 0.999992515713766

In [37]:
result = []
for i in range(len(predictions)):
    if predictions[i][0] >= predictions[i][1]:
        result.append(0)
    else:
        result.append(1)

# Assuming 'dev_data_labels' is a DataFrame column
dev_labels = dev_data_labels.iloc[:len(result)]

count = sum(result[i] == dev_labels.iloc[i] for i in range(len(result)))
accuracy = count / len(result)

print("Accuracy on the development set:", accuracy)


Accuracy on the development set: 0.15310898150211721
